**Lilian JAOUANNE**

**Bastian GARÇON**

Mecatro 3A FISA Promo 2026

# Base de Données sur la Qualité des Vins de Vinho Verde

## Origine des données

Les données sont extraites de l'[UCI Machine Learning Repository](https://archive.ics.uci.edu) et se rapportent spécifiquement aux variantes rouge et blanche des vins de Vinho Verde, une région notable du Portugal.

## Composition de la Base de Données

La base de données se divise en deux ensembles distincts, l'un pour les vins rouges et l'autre pour les vins blancs. Chaque ensemble comprend plusieurs variables qui mesurent les caractéristiques physicochimiques des vins ainsi que leur qualité sensorielle évaluée :
- Fixed Acidity: Acidité totale du vin, incluant les acides fixes et volatils.
- Volatile Acidity: Concentration en acide acétique, influant sur le goût vinaigré du vin.
- Citric Acid: Acidité apportée par l'ajout d'acide citrique, ajustant l'acidité des vins.
- Residual Sugar: Quantité de sucre restant post-fermentation.
- Chlorides: Teneur en chlorures, ou sel, du vin.
- Free Sulfur Dioxide: Quantité de dioxyde de soufre libre, prévenant l'oxydation et la croissance microbienne.
- Total Sulfur Dioxide: Totalité du dioxyde de soufre présent dans le vin.
- Density: Densité du vin, souvent liée à sa teneur en alcool.
- pH: Indice de l'acidité ou basicité du vin.
- Sulphates: Taux de sulfates ajoutés, augmentant le SO2 et aidant à la conservation du vin.
- Alcohol: Pourcentage d'alcool contenu dans le vin.
- Quality: Note de qualité attribuée par des dégustateurs, sur une échelle de 0 à 10.

## Problématiques Principales

### 1. Identification des Variables Influant sur la Densité

- **Objectif** : Détecter les variables physicochimiques qui influencent significativement la densité du vin, pour les vins rouges et blancs.
- **Approche** : Sélection de variables pertinentes à travers des techniques statistiques pour minimiser le bruit et se concentrer sur les contributeurs clés à la densité du vin.

### 2. Modélisation de la Densité du Vin

- **Objectif** : Construire un modèle de régression linéaire multiple pour prédire la densité du vin à partir des variables sélectionnées, et comparer les modèles pour les vins rouges contre les vins blancs.
- **Approche** : Développement et validation d'un modèle de régression, évaluation de la précision et des performances du modèle, et analyse des interactions entre les variables.

### 3. Comparaison des Distributions de Variables Spécifiques

- **Objectif** : Examiner comment les distributions des variables telles que le pH, la densité, et l'acidité volatile varient entre les vins rouges et blancs.
- **Approche** : Utilisation de tests de normalité et d'analyse de variance (ANOVA) pour évaluer les différences statistiques entre les deux types de vin.

### 4. Analyse des Associations entre Qualité du Vin et Autres Variables Catégorielles

- **Objectif** : Investiguer s'il existe des associations significatives entre les catégories de qualité du vin et d'autres variables catégorielles.
- **Approche** : Utilisation de tests statistiques pour évaluer l'association entre variables catégorielles et la qualité perçue du vin.

# Mise en pratique

## Librairies à importer

Avant toute chose, nous devons importer ces librairies :
- _Pandas_ : pour la partie dataframes
- _numpy_ : pour les tableaux
- _pyplot_ de _matplotlib_ : pour une visualisation graphique
- _seaborn_ : pour la visualisation de données
- _statmodels.api_ : pour le test ANOVA
- _shapiro_ de _scipy.stats_ : pour le test Shapiro puis _scipy.stats_ pour les fonctions statistiques.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import shapiro
import scipy.stats as stats

## Importation des bases de données

On importe les deux base de données grâce à _pandas_

In [35]:
RED_WINE = pd.read_csv("Bases de données/winequality-red.csv", sep=';', decimal='.')
WHITE_WINE = pd.read_csv("Bases de données/winequality-white.csv", sep=';', decimal='.')

# 1. Identification des Variables Influant sur la Densité

On commence par vérifier quelles sont les variables corrélées à la densité. Pour cela on utilise la fonction suivante qui va prendre un jeu de données et supprimer les variables n'étant pas corrélées à la densité avec un risque de 5%. Pour cela on utilise la valeur **p** produite par le Test de **Pearson**. Si la valeur **p** est inférieur à 0.05 et la valeur absolue de la corrélation supérieur à 0.15, on garde la variable pour la régression linéaire multiple qui suivra.

In [60]:
def corr_data(data_set, variable:str):
    # Calculer les coefficients de corrélation et les valeurs p pour chaque variable par rapport à 'variable'
    correlations = {}
    p_values = {}
    variables_to_exclude = []

    for column in data_set.columns:
        if column != variable:  # éviter de comparer la varaible à corréler avec elle-même
            corr, p_val = stats.pearsonr(data_set[column], data_set[variable])
            correlations[column] = corr
            p_values[column] = p_val
        
            if abs(corr) < 0.15 or p_val > 0.05:
                variables_to_exclude.append(column)
    
    return data_set.drop(columns = variables_to_exclude)

A présent, nous pouvons appeler la fonction pour le vin rouge et blanc. On oublie d'y enlever la variable de **qualité** qui n'est pas continue. De ce fait, elle est considérée comme une variable catégorielle. On y enlève aussi la **densité** puisque c'est la variable que l'on cherche à prédire.

In [70]:
X_density_RED = corr_data(RED_WINE, 'density')
X_density_RED.drop(columns = ['quality', 'density'], inplace=True)

X_density_WHITE = corr_data(WHITE_WINE, 'density')
X_density_WHITE.drop(columns = ['quality', 'density'], inplace=True)

# On en profite pour mettre de côté les densité
Y_density_RED = RED_WINE['density']
Y_density_WHITE = WHITE_WINE['density']

Maintenant, nous avons notre jeu de donnée avec les variables qui influent le plus sur la densité. On les affiche grâce à la fonction suivante.

In [62]:
print('Variables retenues pour la régression linéaire de la densité du :\nVIN ROUGE :')
print(X_density_RED.keys())
print('\nVIN BLANC :')
print(X_density_WHITE.keys())

Variables retenues pour la régression linéaire de la densité du :
VIN ROUGE :
Index(['fixed acidity', 'citric acid', 'residual sugar', 'chlorides', 'pH',
       'alcohol'],
      dtype='object')

VIN BLANC :
Index(['fixed acidity', 'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'alcohol'],
      dtype='object')


On voit que pour le vin rouge comme pour le vin blanc, les variables concervées pour expliquer la dentité sont :
- fixed acidity
- residual sugar
- chlorides
- alcohol

En revanche les varaibles :
- citric acid
- pH

Ne sont présentent que dans dans les variables expliquatives de la dentité du vin rouge et celle :
- free sulfur dioxide
- total sulfur dioxide

Que dans dans les variables expliquatives de la dentité du vin blanc.

# 2. Modélisation de la Densité du Vin

Créeons notre model grâce à _statmodel_ et les variables que nous avons choisi de garder.

In [63]:
# Ajout d’une constante pour l’intercept
X = sm.add_constant(X_density_RED)
# Ajustement du modèle
model_red_density = sm.OLS(Y_density_RED, X).fit()
# Affichage du résumé du modèle
print(model_red_density.summary())

                            OLS Regression Results                            
Dep. Variable:                density   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     1284.
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        11:39:27   Log-Likelihood:                 9172.4
No. Observations:                1599   AIC:                        -1.833e+04
Df Residuals:                    1592   BIC:                        -1.829e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.9803      0.001   1381.

Après cela, on enlève les occurences qui ont une distance de **Cook** trop importante. De ce fait on évite les valeurs qui ont un levier et un résidue importants. Celle-ci sont particulièrement problématiques puisqu'elles indiquent une mauvaise prédiction et une forte influence sur la ligne de régression.
On utilise le critère stipulant que si la distance de Cook $d_{cook}$ :
$$d_{cook}>\frac{4}{n-k-1}$$
on supprime la valeur correspondante. Avec :
- $n$ : le nombre d'observations
- $k$ : le nombre de variable explicative

Pour cela on calcul les distance de Cook :

In [64]:
influence = model_red_density.get_influence()
d_cooks = influence.cooks_distance[0] # Pour ne garder que la distance de cook
print("Cook's distance:", d_cooks)

Cook's distance: [6.05150788e-05 1.79624078e-04 1.35800997e-04 ... 4.08688424e-04
 3.27885437e-05 9.04164325e-04]


Et on garde les indices des points problématiques.

In [73]:
# Seuil pour la distance de Cook
n = len(d_cooks)
k = len(X_density_RED.keys())
threshold = 4/(n - k - 1)

# Trouver les indices des observations ayant une distance de Cook trop élevée
high_influence_points = np.where(d_cooks > threshold)[0]
print('Nombre de donnée hautement influente : ', len(high_influence_points))

Nombre de donnée hautement influente :  107


On peut à présent filter nos données.

In [71]:
X_density_RED_clean = X_density_RED.drop(index=high_influence_points)
Y_density_RED_clean = Y_density_RED.drop(index=high_influence_points)

Puis refaire la regression.

In [74]:
# Ajout d’une constante pour l’intercept
X = sm.add_constant(X_density_RED_clean)
# Ajustement du modèle
model_red_density = sm.OLS(Y_density_RED_clean, X).fit()
# Affichage du résumé du modèle
print(model_red_density.summary())

                            OLS Regression Results                            
Dep. Variable:                density   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.869
Method:                 Least Squares   F-statistic:                     1651.
Date:                Sat, 08 Jun 2024   Prob (F-statistic):               0.00
Time:                        11:52:36   Log-Likelihood:                 8911.2
No. Observations:                1492   AIC:                        -1.781e+04
Df Residuals:                    1485   BIC:                        -1.777e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.9809      0.001   1561.

On voit que le $R^2$ ajusté est passé de 0.828 à 0.869. Cela signifie que notre model explique à présent 86% des données en supprimant celle qui on un résidu et un levier imporant contre 82% en conservant toutes les données. Ainsi au prix de 107 données, nous avons améliorée le model.

In [75]:
def forward_selection(data, response):
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = float('inf'), float('inf')
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {}".format(response, ' + '.join(selected + [candidate]))
            score = sm.OLS.from_formula(formula, data).fit().aic
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop(0)
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {}".format(response, ' + '.join(selected))
    model = sm.OLS.from_formula(formula, data).fit()
    return model

model = forward_selection(RED_WINE, 'density')
print(model.summary())

SyntaxError: invalid syntax (<unknown>, line 1)